In [ ]:
import numpy as np
import pandas as pd
import xarray as xr
import re
import requests
import holoviews as hv
import matplotlib.pyplot as plt
import pint
import nd2reader
import tifffile
import scipy.stats

In [ ]:
u = pint.UnitRegistry()

In [ ]:
hv.extension("bokeh")

# Setup

In [ ]:
excitation_bins = np.linspace(300, 850, 1000)
emission_bins = excitation_bins

# Sample

In [ ]:
def image_to_xarray(img, scale):
    return xr.DataArray(
        img,
        coords=dict(
            x=scale * np.arange(img.shape[1]), y=scale * np.arange(img.shape[0])[::-1]
        ),
        dims=["y", "x"],
    )

In [ ]:
nd2 = nd2reader.ND2Reader("ClpP_mEGFP_100x.nd2")
# nd2._parser._raw_metadata.z_levels = [0]
sample_img = nd2.get_frame_2D(v=0, c=1)  # [:,500:1000]
sample_img = image_to_xarray(sample_img, nd2.metadata["pixel_microns"])

In [ ]:
sample_img = tifffile.imread("t000001xy14c2.tif")
scale = 4.25 / 20
sample_img = image_to_xarray(sample_img, scale)

In [ ]:
sample_img.plot.imshow()

# Excitation line

In [ ]:
def draw_excitation_line(
    width, edge_defocus, base_defocus, falloff, width_px=6500, height_px=300
):
    if not ((0 <= falloff) and (falloff <= 1)):
        raise ValueError("falloff must be between 0 and 1")
    # expect defocus parameters in um
    width = float(width / u.um)
    edge_defocus = float(edge_defocus / u.um)
    base_defocus = float(base_defocus / u.um)
    x_dependence = np.abs(np.linspace(-1, 1, width_px)) ** 2
    sigma = edge_defocus * x_dependence + base_defocus
    x_max = width / 2
    xs = np.linspace(-x_max, x_max, width_px)
    y_max = 6 * sigma.max()  # 6 sigma
    ys = np.linspace(-y_max, y_max, height_px)
    img = scipy.stats.norm.pdf(
        np.arange(height_px)[:, np.newaxis],
        height_px / 2,
        sigma,
    ) * (1 - falloff * x_dependence)
    return xr.DataArray(img, coords=dict(x=xs, y=ys), dims=["y", "x"])

In [ ]:
line_img = draw_excitation_line(1.3 * u.mm, 2 * u.um, 2 * u.um, 0.3)

In [ ]:
sample_img

In [ ]:
line_img

In [ ]:
sample_img.interp_like(
    line_img.assign_coords(x=line_img.x + 500, y=line_img.y + 470)
).plot(
    aspect=20, size=5
)  # .pcolormesh(aspect="equal", size=10)

In [ ]:
sample_offset = offset_xarray(sample_img, line_img, dict(x=500, y=470))

In [ ]:
sample_offset

In [ ]:
line_img

In [ ]:
(sample_offset * line_img).plot(aspect=20, size=5)

In [ ]:
sample_img.coords

In [ ]:
def offset_xarray(a, b, offsets):
    offsets = {name: getattr(b, name) + val for name, val in offsets.items()}
    return a.interp_like(b.assign_coords(**offsets)).assign_coords(b.coords)

In [ ]:
offset_xarray(sample_img, line_img, dict(x=100, y=30)).plot.imshow()

In [ ]:
a = sample_img.interp_like(line_img.assign_coords(x=line_img.x + 0, y=line_img.y + 20))

In [ ]:
a

In [ ]:
line_img

In [ ]:
a * line_img

In [ ]:
(
    sample_img.interp_like(line_img.assign_coords(x=line_img.x + 0, y=line_img.y + 100))
    * line_img
).plot.imshow()

In [ ]:
line_img * sample_img

In [ ]:
_.plot.imshow()